In [ ]:
from grid.robot.aerial.airgen_drone import AirGenDrone 
airgen_drone__0 = AirGenDrone()

In [ ]:
# Take off
airgen_drone_0.client.takeoffAsync().join()

In [ ]:
import h5py, os
from typing import List
from airgen import ImageType, VehicleClient
from airgen.utils.visualize import rr_log_airgen_image
from grid import GRID_USER_SESSION_BLOB_DIR


def capture_data(drone: VehicleClient, sensor_types: List[str], capture_types: List[str], hdf5_file_path: str, idx=0):
    with h5py.File(hdf5_file_path, 'a') as hdf5_file:
        frame_group = hdf5_file.create_group(f"frame_{idx}")
        sensor_group = frame_group.create_group("sensor_data")
        image_group = frame_group.create_group("image_data")

        capture_name_map = {
            "rgb": ImageType.Scene, 
            "depth": ImageType.DepthPerspective, 
            "segmentation": ImageType.Segmentation
        }
        
        # Capture sensor data
        for sensor_name in sensor_types:
            if sensor_name == "imu":
                imu_data = drone.getImuData()
                imu_group = sensor_group.create_group("imu")
                imu_group.create_dataset("time_stamp", data=imu_data.time_stamp)
                imu_group.create_dataset("orientation", data=[
                    imu_data.orientation.w_val, 
                    imu_data.orientation.x_val, 
                    imu_data.orientation.y_val, 
                    imu_data.orientation.z_val
                ])
                imu_group.create_dataset("angular_velocity", data=[
                    imu_data.angular_velocity.x_val, 
                    imu_data.angular_velocity.y_val, 
                    imu_data.angular_velocity.z_val
                ])
                imu_group.create_dataset("linear_acceleration", data=[
                    imu_data.linear_acceleration.x_val, 
                    imu_data.linear_acceleration.y_val, 
                    imu_data.linear_acceleration.z_val
                ])
            elif sensor_name == "gps":
                gps_loc_data = drone.getGpsData().gnss.geo_point                
                gps_group = sensor_group.create_group("gps")
                gps_group.create_dataset("latitude", data=gps_loc_data.latitude)
                gps_group.create_dataset("longitude", data=gps_loc_data.longitude)
                gps_group.create_dataset("altitude", data=gps_loc_data.altitude)
            else:
                raise ValueError(f"Unknown sensor name: {sensor_name}")

        # Capture image data
        image_types = [capture_name_map[capture_type] for capture_type in capture_types]
        images = drone.getImages("front_center", image_types)
        
        for capture_type, image in zip(capture_types, images):
            rr_log_airgen_image("grid", capture_name_map[capture_type], image[0])
            image_group.create_dataset(capture_type, data=image[0])

    if idx % 10 == 0:
        print(f"Saved {idx} frames")

In [ ]:
import time

#  Example usage
hdf5_file_path = os.path.join(GRID_USER_SESSION_BLOB_DIR, 'drone_data.h5')
start_time = time.time()

# Apply velocity commands for a given duration
airgen_drone_0.client.moveByVelocityAsync(2, 2, -2, 5)

for i in range(50):
    airgen_drone_0.client.simPause(True)
    capture_data(airgen_drone_0.client, ['imu', 'gps'], ['rgb'], hdf5_file_path, idx=i)
    airgen_drone_0.client.simPause(False)
    time.sleep(0.1)

airgen_drone_0.client.hoverAsync().join()

elapsed_time = time.time() - start_time
print(f"Total time taken: {elapsed_time:.2f} seconds")